# COMPETITOR ANALYSIS EDSA INTERNSHIP PROJECT
Explore Data Science wants to develop a marketing startegy to ensure that all social media content posted reaches the inteded target market or audiance and that our social media market are growing in engagement and also analysing our growing social media engagement as compared to the competitors
## This notebook will be used in scraping Our competitors data from LinkedIn webpage 

## A. Importing our projects libraries/dependencies 

In [97]:
# Required Imports
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import time
import pandas as pd
import re as re

## B. Getting our competitors url to get the data from
- By first getting the company page that the user wants to scrape.
- Then by checking for an existing user credentials file or create one if its a first time user

In [404]:
#Get the Company's Linkedin URL from User
# pages = ["https://www.linkedin.com/company/explore-datascience/", 
#          "https://www.linkedin.com/company/simplilearn/",
#          "https://www.linkedin.com/school/hyperion-development-south-africa/", 
#          "https://www.linkedin.com/company/africa-data-school/", 
#          "https://www.linkedin.com/school/moringa-school/", 
#          "https://www.linkedin.com/company/data-science-dojo/", 
#          "https://www.linkedin.com/school/nyc-data-science-academy/", 
#          "https://www.linkedin.com/school/udacity/", 
#          "https://www.linkedin.com/school/getsmarter/"]
# for paged in pages:
#     page = paged

page = input("Enter the Company Linkedin URL: ")
company_name = page[33:-1]

#See if existing user credential file exists or create one 
try:
    f= open("linkedin_credentials.txt","r")
    contents = f.read()
    username = contents.replace("=",",").split(",")[1]
    password = contents.replace("=",",").split(",")[3]
except:
    f= open("linkedin_credentials.txt","w+")
    username = input('Enter your linkedin username: ')
    password = input('Enter your linkedin password: ')
    f.write("username={}, password={}".format(username,password))
    f.close()

Enter the Company Linkedin URL: https://www.linkedin.com/company/simplilearn/


## C. Getting Selenium and ChromeDriver ready
We now need to get Selenium to use ChromeDriver to open chrome and visit Linkedin where it will sign in using your login info

In [405]:
#accessing Chromedriver
browser = webdriver.Chrome('chromedriver')


#Open login page
browser.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')

#Enter login info:
elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)
elementID.submit()

## D. Scrolling Down the Page
We next search for the Linkedin page that we want to scrape. I’m into backpacking and the outdoors so I’ll use REI as an example. Now, this next section of code allows us to scroll down the entire Linkedin page. This is important because with an infinite scroll page such as Linkedin you would only get the first few posts without this step.

In [406]:
#Go to company post webpage
browser.get(page + 'posts/')


#Simulate scrolling to capture all posts
SCROLL_PAUSE_TIME = 1.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

## E. Enter Beautiful Soup…
We will now collect the source code and then starting looking for the page elements that we want to access.

In [407]:
company_page = browser.page_source

#Check out page source code
linkedin_soup = bs(company_page.encode("utf-8"), "html")
linkedin_soup.prettify()

containers = linkedin_soup.findAll("div",{"class":"occludable-update ember-view"})

## F. Finding the elements that we want
It involved locating our target elements and scraping them from our target webpage and laying some conditions that are necessary to avoid any errors by implementing the try and catch methods 

In [408]:
competitors_name = []
post_dates = []
post_texts = []
post_likes = []
post_comments = []
post_shared = []
# video_views = []
# media_links = []
media_type = []
total_followers = []

for container in containers:

        try:
            competitor = container.find("span", {"class":"feed-shared-actor__name"})
            posted_date = container.find("span",{"class":"visually-hidden"})
            text_box = container.find("div",{"class":"feed-shared-update-v2__description-wrapper"})
            text = text_box.find("span",{"dir":"ltr"})
            post_shares = container.findAll("li", {'class':["social-details-social-counts__item--with-social-proof","social-details-social-counts__comments social-details-social-counts__item"]})
            new_likes = container.findAll("li", {'class':["social-details-social-counts__item--with-social-proof","social-details-social-counts__item"]})
            new_comments = container.findAll("li", {"class": "social-details-social-counts__item social-details-social-counts__comments social-details-social-counts__item--with-social-proof"})
            followers = container.find("span", {"class":"feed-shared-actor__description"})

            competitors_name.append(competitor.text.strip())
            post_dates.append(posted_date.text.strip())
            post_texts.append(text.text.strip())
            total_followers.append(followers.text.strip())



            try:
                video_box = container.findAll("div",{"class": "feed-shared-update-v2__content feed-shared-linkedin-video ember-view"})
                video_link = video_box[0].find("video", {"class":"vjs-tech"})
                media_links.append(video_link['src'])
                media_type.append("Video")
            except:
                try:
                    image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                    image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image feed-shared-image__image--constrained lazy-image ember-view"})
                    media_links.append(image_link['src'])
                    media_type.append("Image")
                except:
                    try:
                        #mutiple shared images
                        image_box = container.findAll("div",{"class": "feed-shared-image__container"})
                        image_link = image_box[0].find("img", {"class":"ivm-view-attr__img--centered feed-shared-image__image lazy-image ember-view"})
                        media_links.append(image_link['src'])
                        media_type.append("Multiple Images")
                    except:
                        try:
                            article_box = container.findAll("div",{"class": "feed-shared-article__description-container"})
                            article_link = article_box[0].find('a', href=True)
                            media_links.append(article_link['href'])
                            media_type.append("Article")
                        except:
                            try:
                                video_box = container.findAll("div",{"class": "feed-shared-external-video__meta"})          
                                video_link = video_box[0].find('a', href=True)
                                media_links.append(video_link['href'])
                                media_type.append("Youtube Video")   
                            except:
                                try:
                                    poll_box = container.findAll("div",{"class": "feed-shared-update-v2__content overflow-hidden feed-shared-poll ember-view"})
                                    media_links.append("None")
                                    media_type.append("Other: Poll, Shared Post, etc")
                                except:
                                    media_links.append("None")
                                    media_type.append("Unknown")

            try:
                post_likes.append(new_likes[0].text.strip())
            except:
                post_likes.append(0)
                pass

            try:
                post_comments.append(new_comments[0].text.strip())                           
            except:                                                           
                post_comments.append("0")
                pass

            try:
                post_shared.append(post_shares[1].text.strip())                           
            except:                                                           
                post_shared.append("0")
                pass

        except:
            pass

## G. Cleaning our scraped data

#### 1. Viewing our competitors name on the platform

In [409]:
print(competitors_name)

['Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn', 'Simplilearn']


#### 2. Getting the total numbers of followers for our competitors

In [410]:
# print(total_followers)
followers_clean_up = []

for i in total_followers:
    c = str(i[0:-9]).replace('followers','').replace(' ', '')
    followers_clean_up += [c]

#### 3. Viewing our followers count on the platform

In [411]:
print(followers_clean_up)

['330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644', '330,644']


### i. Cleaning our date of the post column

#### 1. Remove any special characters that might have been scraped during our initial process that are unnecessary in our use case

In [412]:
cleaned_dates = []
for i in post_dates:
    d = str(i[0:3]).replace('\n\n', '').replace('•','').replace(' ', '')
    cleaned_dates += [d]

In [413]:
print(cleaned_dates)

['2d', '6d', '1d', '4d', '3d', '2w', '5d', '3w', '1m', '1w', '1w', '1w', '1w', '1m', '2w', '2w', '2w', '2m', '1m', '1m', '2w', '2m', '1m', '3w', '2m', '4m', '3w', '1m', '3m']


#### 1.1 Getting our current time 

In [414]:
from datetime import datetime

# current date and time
now = datetime.today()
print(now)

2022-04-25 10:39:36.331523


#### 2. A logic to convert our no of days, no of weeks, no of months and years into a timestamp
The following logic is making use of the dateutil module known as relativedelta to convert our hours, days, months and years scraped in our data into actual datetime objects in order to know the exact date and it's precise time through which a post was made by an organization

In [415]:
import datetime
from dateutil.relativedelta import relativedelta

posts_posted_date = []

for clean_d in cleaned_dates:
    clean_d.split()
    if clean_d[-1] == "h":
        t_hour = int(clean_d[0])
        posts_posted_date.append(now - relativedelta(hours = t_hour))
    elif clean_d[-1] == "d":
        t_day = datetime.timedelta(days = int(clean_d[0]))
        posts_posted_date.append(now - t_day)
    elif clean_d[-1] == "w":
        t_week = datetime.timedelta(days = int(clean_d[0]) * 7)
        posts_posted_date.append(now - t_week)
    elif clean_d[-1] == "m":
        t_month = int(clean_d[0])
        posts_posted_date.append(now - relativedelta(months = t_month))
    elif clean_d == "y":
        t_year = int(clean_d[0])
        posts_posted_date.append(now - relativedelta(years = t_year))
    else:
#         posts_posted_date.append(timestamp - (int(clean_d) * 60 * 1000))
        posts_posted_date.append(now - relativedelta(minutes= 30))

#### 3. Printing our date of the posts

In [416]:
print(posts_posted_date)

[datetime.datetime(2022, 4, 23, 10, 39, 36, 331523), datetime.datetime(2022, 4, 19, 10, 39, 36, 331523), datetime.datetime(2022, 4, 24, 10, 39, 36, 331523), datetime.datetime(2022, 4, 21, 10, 39, 36, 331523), datetime.datetime(2022, 4, 22, 10, 39, 36, 331523), datetime.datetime(2022, 4, 11, 10, 39, 36, 331523), datetime.datetime(2022, 4, 20, 10, 39, 36, 331523), datetime.datetime(2022, 4, 4, 10, 39, 36, 331523), datetime.datetime(2022, 3, 25, 10, 39, 36, 331523), datetime.datetime(2022, 4, 18, 10, 39, 36, 331523), datetime.datetime(2022, 4, 18, 10, 39, 36, 331523), datetime.datetime(2022, 4, 18, 10, 39, 36, 331523), datetime.datetime(2022, 4, 18, 10, 39, 36, 331523), datetime.datetime(2022, 3, 25, 10, 39, 36, 331523), datetime.datetime(2022, 4, 11, 10, 39, 36, 331523), datetime.datetime(2022, 4, 11, 10, 39, 36, 331523), datetime.datetime(2022, 4, 11, 10, 39, 36, 331523), datetime.datetime(2022, 2, 25, 10, 39, 36, 331523), datetime.datetime(2022, 3, 25, 10, 39, 36, 331523), datetime.dat

### ii. Cleaning our comment count 

#### 1. Viewing our comment counts raw data

In [417]:
print(post_comments)

['1 comment', '5 comments', '1 comment', '1 comment', '2 comments', '21 comments', '134 comments', '2 comments', '3 comments', '6 comments', '0', '5 comments', '0', '2 comments', '0', '2 comments', '2 comments', '0', '0', '16 comments', '2 comments', '1 comment', '0', '0', '14 comments', '5 comments', '1 comment', '33 comments', '3 comments']


#### 2. Iterating over and trimming our comments counts and removing unnecesarry whitespaces

In [418]:
posts_counted_comments = []
for cm in post_comments:
    cm.split()
    if len(cm) > 2:
        posts_counted_comments.append(cm[0:2].strip())
    else:
        posts_counted_comments.append(0)

#### 3. Viewing our cleaned comments counts

In [419]:
print(posts_counted_comments)

['1', '5', '1', '1', '2', '21', '13', '2', '3', '6', 0, '5', 0, '2', 0, '2', '2', 0, 0, '16', '2', '1', 0, 0, '14', '5', '1', '33', '3']


### iii. Cleaning our shared counts column

#### 1. Viewing our raw share count row data collected

In [420]:
print(post_shared)

['3 shares', '2 shares', '0', '1 share', '4 shares', '9 shares', '4 shares', '1 share', '1 share', '2 shares', '0', '0', '0', '4 shares', '0', '0', '1 share', '0', '0', '6 shares', '0', '0', '0', '0', '0', '0', '0', '2 shares', '0']


#### 2. Cleaning our share count row

In [421]:
posts_counted_shares = []
for ps in post_shared:
    ps.split()
    if len(ps) > 2:
        posts_counted_shares.append(ps[0:2])
    else:
        posts_counted_shares.append(0)

#### 3. Viewing our cleaned share count column

In [422]:
print(posts_counted_shares)

['3 ', '2 ', 0, '1 ', '4 ', '9 ', '4 ', '1 ', '1 ', '2 ', 0, 0, 0, '4 ', 0, 0, '1 ', 0, 0, '6 ', 0, 0, 0, 0, 0, 0, 0, '2 ', 0]


## H. Organizing the Data for Export
Finally, we are ready to organize our collected data into a Pandas DataFrame and then export it as a CSV or Excel file. You can find this file saved in the same folder that you ran the program from

In [423]:
data = {
    "our_competitors": competitors_name,
    "Date_Posted": posts_posted_date,
    "Media_Type": media_type,
    "Post_Caption": post_texts,
    "Likes_Counts": post_likes,
    "Comments_Counts": posts_counted_comments,
    "Shared_Counts" : posts_counted_shares,
    "total_followers": followers_clean_up
#     "Video Views": video_views,
#     "Media Links": media_links
}

df = pd.DataFrame(data)
df

,our_competitors,Date_Posted,Media_Type,Post_Caption,Likes_Counts,Comments_Counts,Shared_Counts,total_followers
0,Simplilearn,2022-04-23 10:39:36.331523,"Other: Poll, Shared Post, etc",What a fantastic end to the week!Here is a sne...,197,1,3,"330,644"
1,Simplilearn,2022-04-19 10:39:36.331523,"Other: Poll, Shared Post, etc","Yes, that’s right, but…😅Which one do you relat...",158,5,2,"330,644"
2,Simplilearn,2022-04-24 10:39:36.331523,Image,🙌 Here’s to more wins!We all know that this is...,50,1,0,"330,644"
3,Simplilearn,2022-04-21 10:39:36.331523,"Other: Poll, Shared Post, etc",Google Ads is sunsetting the old expanded text...,11,1,1,"330,644"
4,Simplilearn,2022-04-22 10:39:36.331523,"Other: Poll, Shared Post, etc",Pull shot for a six?No Guarantee.But with Simp...,173,2,4,"330,644"
5,Simplilearn,2022-04-11 10:39:36.331523,"Other: Poll, Shared Post, etc",#HiringAlertWe are looking for qualified Instr...,78,21,9,"330,644"
6,Simplilearn,2022-04-20 10:39:36.331523,"Other: Poll, Shared Post, etc",Webinar - How to Build a Personal Brand and Un...,103,13,4,"330,644"
7,Simplilearn,2022-04-04 10:39:36.331523,"Other: Poll, Shared Post, etc",We just concluded the second online convocatio...,29,2,1,"330,644"
8,Simplilearn,2022-03-25 10:39:36.331523,"Other: Poll, Shared Post, etc",The 2nd edition of the Data-Driven Government ...,102,3,1,"330,644"
9,Simplilearn,2022-04-18 10:39:36.331523,Image,Upskilling can be your biggest flex and the on...,195,6,2,"330,644"


In [424]:
df.head(50)

,our_competitors,Date_Posted,Media_Type,Post_Caption,Likes_Counts,Comments_Counts,Shared_Counts,total_followers
0,Simplilearn,2022-04-23 10:39:36.331523,"Other: Poll, Shared Post, etc",What a fantastic end to the week!Here is a sne...,197,1,3,"330,644"
1,Simplilearn,2022-04-19 10:39:36.331523,"Other: Poll, Shared Post, etc","Yes, that’s right, but…😅Which one do you relat...",158,5,2,"330,644"
2,Simplilearn,2022-04-24 10:39:36.331523,Image,🙌 Here’s to more wins!We all know that this is...,50,1,0,"330,644"
3,Simplilearn,2022-04-21 10:39:36.331523,"Other: Poll, Shared Post, etc",Google Ads is sunsetting the old expanded text...,11,1,1,"330,644"
4,Simplilearn,2022-04-22 10:39:36.331523,"Other: Poll, Shared Post, etc",Pull shot for a six?No Guarantee.But with Simp...,173,2,4,"330,644"
5,Simplilearn,2022-04-11 10:39:36.331523,"Other: Poll, Shared Post, etc",#HiringAlertWe are looking for qualified Instr...,78,21,9,"330,644"
6,Simplilearn,2022-04-20 10:39:36.331523,"Other: Poll, Shared Post, etc",Webinar - How to Build a Personal Brand and Un...,103,13,4,"330,644"
7,Simplilearn,2022-04-04 10:39:36.331523,"Other: Poll, Shared Post, etc",We just concluded the second online convocatio...,29,2,1,"330,644"
8,Simplilearn,2022-03-25 10:39:36.331523,"Other: Poll, Shared Post, etc",The 2nd edition of the Data-Driven Government ...,102,3,1,"330,644"
9,Simplilearn,2022-04-18 10:39:36.331523,Image,Upskilling can be your biggest flex and the on...,195,6,2,"330,644"


## I. Saving our scrapped data in a CSV file

In [425]:
#Exporting as csv file to program folder
df.to_csv("{}.csv".format(company_name), encoding='utf-8', index=False)

## J. Saving our scrapped data in a Spreadsheet file

In [426]:
#Export to Excel file to program folder
writer = pd.Excelwriter = pd.ExcelWriter("{}.xlsx".format(company_name), engine='xlsxwriter')

df.to_excel(writer, index =False)
writer.save()

## Conclusion